In [15]:
from couplets_utils import *
from keras.layers import *
from keras.models import Model
from keras import backend as K
import numpy as np
import tensorflow as tf

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
N_A = 128

In [17]:
sample = load_sample_datasets(vocabs_size=2000, max_len=30, batch_size=16, sample_size=1000, n_a=N_A)
sample_gen = sample['sample_gen']
index2word = sample['index2word']
word2index = sample['word2index']
vocabs_size = sample['vocabs_size']
max_len = sample['max_len']

In [18]:
print('sample_gen length :', len(sample_gen))

sample_gen length : 63


In [19]:
def create_train_model(n_x, n_a, Tx, keep_prob=0.5):    
    input = Input(shape=(Tx, n_x), name='x0')
    a_0_0 = Input(shape=(n_a,), name='a_0_0')
    c_0_0 = Input(shape=(n_a,), name='c_0_0')
    a_1_0 = Input(shape=(n_a,), name='a_1_0')
    c_1_0 = Input(shape=(n_a,), name='c_1_0')
        
    a_0 = a_0_0
    c_0 = c_0_0
    a_1 = a_1_0
    c_1 = c_1_0
    
    lstm_cell_0 = LSTM(units=n_a, return_state=True, name='lstm_0')
    lstm_cell_1 = LSTM(units=n_a, return_state=True, name='lstm_1')
    dense_layer_2 = Dense(units=n_x, activation='softmax', name='softmax_2')
        
    outputs = []
        
    for i in range(Tx):
        x = Lambda(lambda j: j[:, i, :])(input)
        x = Reshape(target_shape=(1, -1))(x)
        a_0, x, c_0 = lstm_cell_0(x, initial_state=[a_0, c_0])
        x = Dropout(rate=1-keep_prob)(x)
        x = Reshape(target_shape=(1, -1))(x)
        a_1, x, c_1 = lstm_cell_1(x, initial_state=[a_1, c_1])
        x = Dropout(rate=1-keep_prob)(x)
        x = dense_layer_2(x)
        outputs.append(x)
                            
    model = Model(inputs=[input, a_0_0, c_0_0, a_1_0, c_1_0], outputs=outputs)
        
    return model

In [20]:
train_model = create_train_model(vocabs_size, N_A, max_len)

In [21]:
train_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [23]:
result = train_model.fit_generator(sample_gen, epochs=1)

Epoch 1/1
63/63 [==============================] - 40s 633ms/step - loss: 156.5443 - softmax_2_loss_1: 7.5616 - softmax_2_loss_2: 7.5211 - softmax_2_loss_3: 7.4108 - softmax_2_loss_4: 7.2242 - softmax_2_loss_5: 6.8566 - softmax_2_loss_6: 7.0495 - softmax_2_loss_7: 7.7294 - softmax_2_loss_8: 5.9416 - softmax_2_loss_9: 7.6246 - softmax_2_loss_10: 7.5361 - softmax_2_loss_11: 7.4984 - softmax_2_loss_12: 7.0458 - softmax_2_loss_13: 6.4441 - softmax_2_loss_14: 6.7139 - softmax_2_loss_15: 6.5470 - softmax_2_loss_16: 4.9805 - softmax_2_loss_17: 3.9223 - softmax_2_loss_18: 3.4926 - softmax_2_loss_19: 3.8836 - softmax_2_loss_20: 3.6381 - softmax_2_loss_21: 3.6753 - softmax_2_loss_22: 3.5416 - softmax_2_loss_23: 3.5942 - softmax_2_loss_24: 3.4512 - softmax_2_loss_25: 3.3751 - softmax_2_loss_26: 2.8484 - softmax_2_loss_27: 2.5258 - softmax_2_loss_28: 2.3722 - softmax_2_loss_29: 2.2865 - softmax_2_loss_30: 2.2523 - softmax_2_acc_1: 0.0000e+00 - softmax_2_acc_2: 0.0000e+00 - softmax_2_acc_3: 9.9219e

In [25]:
dir(result)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'epoch',
 'history',
 'model',
 'on_batch_begin',
 'on_batch_end',
 'on_epoch_begin',
 'on_epoch_end',
 'on_train_begin',
 'on_train_end',
 'params',
 'set_model',
 'set_params',
 'validation_data']

In [9]:
train_model.save_weights('./weights.h5')

In [11]:
infer_model = create_infer_model(vocabs_size, N_A, max_len)
infer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
infer_model.load_weights('./weights.h5')

In [13]:
def write_couplets(begin_text, infer_model):
    x = convert_text_to_onehot(begin_text, vocabs_size, max_len, word2index)
    a_0 = np.zeros((1, N_A))
    c_0 = np.zeros((1, N_A))
    a_1 = np.zeros((1, N_A))
    c_1 = np.zeros((1, N_A))
    
    result = infer_model.predict([x, a_0, c_0, a_1, c_1])
    result_text = convert_predict_to_text(np.array(result), index2word)
    
    return result_text

In [14]:
text = '天天向上'
result_text = write_couplets(text, infer_model)
print(result_text)

天天向上                         
